In [1]:

is_auto_eda = False # run sweetviz report
is_small_spacy = True
is_sample_df = True
sample_fraction = 0.1 # fraction of sampling from train and test DataFrames
num_epochs = 2

In [2]:

# needed for BERT model
!pip install -qq transformers==4.28.0  # (-qq is for quite)

In [4]:
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 kB 3.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

import spacy
import spacy.cli
from spacy.lang.en.stop_words import STOP_WORDS

# for spaCy Text Categorizer
from spacy.util import minibatch, compounding
from spacy.training.example import Example

from wordcloud import WordCloud # See : https://www.kaggle.com/aashita/word-clouds-of-various-shapes

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

from transformers import BertTokenizer, BertForSequenceClassification

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn.functional as F # for softmax in Eval

from datetime import datetime
import pytz # timezone

from tqdm import trange # used to make our loops show a smart progress meter

# from autocorrect import spell # (TODO: do we really need this?)

#torch.manual_seed(0)

print("Using PyTorch version", torch.__version__)

/Users/chandramaniyadav/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using PyTorch version 2.0.1


In [6]:

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns",100)
pd.set_option("display.max_colwidth", 500)
color = sns.color_palette()

In [7]:

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu") # (TODO: change "cpu" to "device error"?)
device

No GPU available, using the CPU instead.


device(type='cpu')

In [8]:

if is_small_spacy:
  spacy.cli.download("en_core_web_sm")
  nlp = spacy.load('en_core_web_sm')
else:
  spacy.cli.download("en_core_web_lg")
  nlp = spacy.load('en_core_web_lg')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 6.5 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [24]:
df = pd.read_csv('/Users/chandramaniyadav/Desktop/My Research Works/Cheating Detection with Generative AI/df_main.csv')
print(df.shape)

(3428, 2)


In [25]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3428 entries, 0 to 3427
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3415 non-null   object
 1   label   3428 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 53.7+ KB


In [26]:
df['text'] = df['text'].fillna('N/A')
len(df)

3428

In [29]:
duplicated_rows = df[df.duplicated(keep=False)]
duplicated_rows = df[df.duplicated(keep=False)]
print(duplicated_rows)

      text  label
470    N/A      1
998    N/A      1
1014   N/A      1
1297   N/A      1
1430   idk      1
1525   N/A      1
1542   N/A      1
1689   N/A      1
1739   N/A      1
1760  nbsp      1
2153   idk      1
2551  nbsp      1
2786   N/A      1
3021   N/A      1
3205   N/A      1
3216  nbsp      1
3220   N/A      1
3397   N/A      1


In [14]:
df = df.drop_duplicates()

In [15]:
df.describe(include='all')

,text,label
count,3413,3413.000000
unique,3413,NaN
top,nbspnbspnbspnbspnbspnbspnbspin shakespeares hamlet the king of denmark is dead his brother has married his wife and taken the throne hamlet the kings son is in mourning but soon believes his uncle to be the murderer of his father in the play hamlet is driven more and more mad through his plots and paranoia and romance hamlet is left out of place as a chachter hamlet was left a madmad with a vendetta against his uncle who he must have vengence againstit may be difficult to comprehend hamlet b...,NaN
freq,1,NaN
mean,NaN,0.853501
std,NaN,0.353657
min,NaN,0.000000
25%,NaN,1.000000
50%,NaN,1.000000
75%,NaN,1.000000


In [16]:

if is_auto_eda:
  !pip install sweetviz

  import sweetviz as sv

  eda_report = sv.analyze(df) #, pairwise_analysis='off')
  eda_report.show_html('NLP_Toxic_Comments_sweetviz.html')

In [17]:
df_orig = df.copy()

In [18]:

# example:
doc = nlp("These are my words of one sentence.")
for token in doc:
  print(token.text, token.pos_, token.dep_)

These PRON nsubj
are AUX ROOT
my PRON poss
words NOUN attr
of ADP prep
one NUM nummod
sentence NOUN pobj
. PUNCT punct


In [19]:
# list of Doc objects
%%time
df['preprocessed_text'] = list(nlp.pipe(df['text']))

UsageError: Line magic function `%%time` not found.


In [20]:
def clean_doc(doc):
  list_clean_tokens =  [token for token in doc if
              not token.is_punct # punctuation
              and not token.is_currency
              and not token.is_digit
              # and not token.is_oov # Is the token out-of-vocabulary (i.e. does it not have a word vector)?
              and not token.is_space # Does the token consist of whitespace characters? Equivalent to token.text.isspace().
              and not token.is_stop
              and not token.like_num
              and not token.like_url and ('@' not in token.text) and ('|' not in token.text)
              # and not token.pos_ == "PROPN" ("Wikipedia")
              ]
  return list_clean_tokens

# Lemmatizing + to lower case
def lemma_text(doc):
    tokens=[]
    for token in doc:
        if token.lemma_ != "-PRON-":
            lemma = token.lemma_.lower().strip()
        else:
            lemma = token.lower_
        tokens.append(lemma)
    return tokens

#create_string
def create_string(doc):
  new_string = ' '.join([str(token) for token in doc])
  return new_string

In [21]:
def preprocess_all(df):
    df['preprocessed_text'] = df['preprocessed_text'].apply(clean_doc)
    df['preprocessed_text'] = df['preprocessed_text'].apply(lemma_text)
    df['preprocessed_text'] = df['preprocessed_text'].apply(create_string)
    return df

In [22]:
df_preprocessed = preprocess_all(df)

KeyError: 'preprocessed_text'

In [33]:
df['label'].value_counts()

1    2928
0     500
Name: label, dtype: int64

In [ ]:

def plot_wordcloud(text, mask=None, figure_size=(15.0,10.0),
                   title = None, title_size=40, image_color=False):

    wordcloud = WordCloud(stopwords=STOP_WORDS,background_color='white',
                    random_state = 42,
                    width=800,
                    height=400,
                    mask = mask,
                    collocations=False) # Do not repeat words in the cloud
    wordcloud.generate(str(text))

    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'black',
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()

In [ ]:
plot_wordcloud(df_preprocessed[df_preprocessed['label']==0]['preprocessed_text'], title="Word Cloud of Generated essay")

In [ ]:
df.to_csv('/content/drive/MyDrive/Cheating Detection for AI generated/df_with_preprocess.csv')

In [ ]:

X = df_preprocessed['preprocessed_text']
y = df_preprocessed['label']

In [ ]:
X.shape

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
np.unique(y_val, return_counts = True)

In [ ]:

# Prepare data
train_texts = X_train.to_list()  # List of training text samples
train_labels = y_train.to_list()  # List of training labels (0 for non-toxic, 1 for toxic)

val_texts = X_val.to_list()  # List of val text samples
val_labels = y_val.to_list()

In [ ]:
from sklearn.metrics import cohen_kappa_score, roc_auc_score

In [ ]:
def get_fp_score(CM) :
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]

    print (TN, FP)

    return FP / (FP + TN)

In [ ]:
results_df = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'QWK', 'F1-score', 'Confusion Matrix', 'FP Score'])



In [ ]:
def store_results(model_name, y_true, y_pred, target_names):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic')



    report = classification_report(y_true, y_pred, target_names=target_names)
    cm = confusion_matrix(y_true, y_pred)
    false_positive = get_fp_score(cm)

    results_df.loc[model_name] = [accuracy, precision, recall,qwk, f1, cm, false_positive]

In [ ]:

target_names = ['Generated', 'Non-Generated']

def print_results(y_true, y_pred):
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)

  print(f'Accuracy: {accuracy:.4f}')
  print(f'Precision: {precision:.4f}')
  print(f'Recall: {recall:.4f}')
  print(f'F1-score: {f1:.4f}')

  # Classification Report
  report = classification_report(y_true, y_pred, target_names=target_names)
  print(f"\nClassification Report:\n{report}")

  # Confusion Matrix
  cm = confusion_matrix(y_true, y_pred)
  print('Confusion Matrix:')
  print(cm)

In [ ]:
max_features=30000

vectorizer = TfidfVectorizer(max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)


In [ ]:
df = pd.DataFrame(data=X_train_vec.todense(), columns=vectorizer.get_feature_names_out())
print(df)

In [ ]:
X_train_vec

In [ ]:
df.to_csv('/content/drive/MyDrive/Cheating Detection for AI generated/Features.csv', index = False)

In [ ]:

# Train the logistic regression classifier
log_reg_classifier = LogisticRegression()
log_reg_classifier.fit(X_train_vec, y_train)

# Predict on the val set
y_pred = log_reg_classifier.predict(X_val_vec)

# Evaluate the model
print("Logistic Regression:")
print_results(y_val, y_pred)
store_results('Logistic Regression', y_val, y_pred, target_names)


In [ ]:
results_df

In [ ]:
# Train the LinearSVC classifier
svc_classifier = LinearSVC() # (TODO: random_state=42, dual=False, max_iter=1000)
svc_classifier.fit(X_train_vec, y_train)

# Predict on the val set
y_pred = svc_classifier.predict(X_val_vec)

# Evaluate the model
print("LinearSVC:")
print_results(y_val, y_pred)
store_results('SVM', y_val, y_pred, target_names)


In [ ]:
# Train the Multinomial Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

# Predict on the val set
y_pred = nb_classifier.predict(X_val_vec)

# Evaluate the model
print("Multinomial Naive Bayes:")
print_results(y_val, y_pred)
store_results('Naive Bayes', y_val, y_pred, target_names)

In [ ]:
results_df

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Train the Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train_vec, y_train)

# Predict on the validation set
y_pred_gb = gb_classifier.predict(X_val_vec)

# Evaluate the Gradient Boosting model
print("Gradient Boosting:")
print_results(y_val, y_pred_gb)
store_results('Gradient Boosting', y_val, y_pred_gb, target_names)

# Train the Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_vec, y_train)

# Predict on the validation set
y_pred_rf = rf_classifier.predict(X_val_vec)

# Evaluate the Random Forest model
print("Random Forest:")
print_results(y_val, y_pred_rf)
store_results('Random Forest', y_val, y_pred_rf, target_names)

# Train the Decision Tree classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_vec, y_train)

# Predict on the validation set
y_pred_dt = dt_classifier.predict(X_val_vec)

# Evaluate the Decision Tree model
print("Decision Tree:")
print_results(y_val, y_pred_dt)
store_results('Decision Tree', y_val, y_pred_dt, target_names)


In [ ]:
results_df

In [ ]:

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
MAX_LEN = 512 # (TODO: change to 128? or something else?)
BATCH_SIZE = 16
NUM_CLASSES = 2

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case = True)

In [ ]:

max_sentence_len_tokens = 0
input_ids_lens = []

# For every sentence...
for sentence in df_preprocessed['preprocessed_text']:

    # Tokenize the text and add `[CLS]` (sentence start), and `[SEP]` (sentence end) tokens.
    input_ids = tokenizer.encode(sentence, add_special_tokens = True) # (TODO: add max_length=512, truncation=True if needed)

    # append to list of lens
    input_ids_lens.append(len(input_ids))

    # Update the maximum sentence length.
    max_sentence_len_tokens = max(max_sentence_len_tokens, len(input_ids))

print('Max sentence length (tokens):', max_sentence_len_tokens)
print('Total number of tokens in all sentences (including special tokens):', sum(input_ids_lens))

# Old - Seaborn
# sns.histplot(input_ids_lens)
# plt.xlim([0, 400])
# plt.xlabel('Number of tokens in a sentence')
# plt.title("Num tokens Histogram");

# Use Plotly instead of Seaborn
fig = go.Figure(data=[go.Histogram(x=input_ids_lens)])
fig.update_layout(
    xaxis=dict(title='Number of tokens in a sentence'),
    yaxis=dict(title='Count'),
    title="Num tokens Histogram",
    bargap=0.1
)
fig.show()

In [ ]:
# Tokenize and encode the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LEN)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LEN)

In [ ]:
class EssayDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, index):
        item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[index])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:

train_dataset = EssayDataset(train_encodings, train_labels)
val_dataset = EssayDataset(val_encodings, val_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME,
    num_labels = NUM_CLASSES,
    output_attentions = False,
    output_hidden_states = False,
)

# we don't need this really: model.classifier = nn.Linear(in_features=model.config.hidden_size, out_features=NUM_CLASSES)

model = model.to(device)

print(model)

In [ ]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-08) # (TODO: play with hyperparameters: lr = 5e-5, eps = 1e)

loss_func = nn.CrossEntropyLoss()

In [ ]:
num_epochs = 4

In [ ]:

%%time
model.train()  # Set the model to training mode


for epoch in trange(num_epochs): # trange - used to make our loops show a smart progress meter
    time_str = datetime.now()
    print(time_str)
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        #loss = outputs.loss (this line was before using loss_func)
        logits = outputs.logits
        loss = loss_func(logits, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}')

In [ ]:
def evaluate(data_loader):
  # Put the model into evaluation mode which disables dropout and batch normalization layers
  model.eval()
  '''
  After the training loop, you can call model.eval() to switch the model to evaluation mode.
  In evaluation mode, the behavior of dropout and batch normalization layers changes.
  Dropout layers no longer drop inputs, and batch normalization layers use the running statistics computed during training instead of calculating batch-wise statistics.
  It's essential to set the model to the appropriate mode (train() or eval()),
  as it ensures that the correct behavior is applied to the model's modules during training and evaluation/testing, respectively.
  '''
  y_true = []
  y_pred = []

  # The torch.no_grad() context ensures that gradients are not computed during testing, which reduces memory usage and speeds up inference
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask)

      # this line was before using softmax
      #_, predicted = torch.max(outputs.logits, dim=1)

      # Obtain the model's logits
      logits = outputs.logits

      # Add softmax activation function to obtain the probabilities for each class.
      # Note that softmax is typically used during inference or evaluation to obtain class probabilities.
      # During training, the model usually directly outputs logits, and the loss function handles the conversion to probabilities.
      probabilities = F.softmax(logits, dim=1) # see https://www.pinecone.io/learn/train-sentence-transformers-softmax/

      # Use torch.max to get the predicted labels based on the highest probability
      _, predicted = torch.max(probabilities, dim=1)

      # 'labels' and 'predicted' are tensors, so we cast them to numpy arrays
      y_true.extend(labels.cpu().numpy())
      y_pred.extend(predicted.cpu().numpy())

  print("BERT model results:")
  print_results(y_true, y_pred)

  return y_true, y_pred

In [ ]:
%%time
y_true, y_pred = evaluate(val_loader)

In [ ]:
store_results('Bert Model', y_true, y_pred, target_names)

In [ ]:
results_df

In [ ]:
results_df.to_csv("/content/drive/MyDrive/Cheating Detection for AI generated/Results of models.csv")

## Electra

In [ ]:
results_df = pd.read_csv('/content/drive/MyDrive/Cheating Detection for AI generated/Results of models.csv')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:

PRE_TRAINED_MODEL_NAME_ELECTRA = 'google/electra-base-discriminator'
MAX_LEN = 512 # (TODO: change to 128? or something else?)
BATCH_SIZE = 16
NUM_CLASSES = 2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME_ELECTRA, do_lower_case = True)

In [ ]:
# Tokenize and encode the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LEN)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LEN)

In [ ]:

train_dataset = EssayDataset(train_encodings, train_labels)
val_dataset = EssayDataset(val_encodings, val_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME_ELECTRA,
    num_labels = NUM_CLASSES,
    output_attentions = False,
    output_hidden_states = False,
)

# we don't need this really: model.classifier = nn.Linear(in_features=model.config.hidden_size, out_features=NUM_CLASSES)

model = model.to(device)
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-08) # (TODO: play with hyperparameters: lr = 5e-5, eps = 1e)

loss_func = nn.CrossEntropyLoss()


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:

%%time
model.train()  # Set the model to training mode

num_epochs = 4
for epoch in trange(num_epochs): # trange - used to make our loops show a smart progress meter
    time_str = datetime.now()
    print(time_str)
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        #loss = outputs.loss (this line was before using loss_func)
        logits = outputs.logits
        loss = loss_func(logits, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}')

In [ ]:
def evaluate(data_loader):
  # Put the model into evaluation mode which disables dropout and batch normalization layers
  model.eval()
  '''
  After the training loop, you can call model.eval() to switch the model to evaluation mode.
  In evaluation mode, the behavior of dropout and batch normalization layers changes.
  Dropout layers no longer drop inputs, and batch normalization layers use the running statistics computed during training instead of calculating batch-wise statistics.
  It's essential to set the model to the appropriate mode (train() or eval()),
  as it ensures that the correct behavior is applied to the model's modules during training and evaluation/testing, respectively.
  '''
  y_true = []
  y_pred = []

  # The torch.no_grad() context ensures that gradients are not computed during testing, which reduces memory usage and speeds up inference
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask)

      # this line was before using softmax
      #_, predicted = torch.max(outputs.logits, dim=1)

      # Obtain the model's logits
      logits = outputs.logits

      # Add softmax activation function to obtain the probabilities for each class.
      # Note that softmax is typically used during inference or evaluation to obtain class probabilities.
      # During training, the model usually directly outputs logits, and the loss function handles the conversion to probabilities.
      probabilities = F.softmax(logits, dim=1) # see https://www.pinecone.io/learn/train-sentence-transformers-softmax/

      # Use torch.max to get the predicted labels based on the highest probability
      _, predicted = torch.max(probabilities, dim=1)

      # 'labels' and 'predicted' are tensors, so we cast them to numpy arrays
      y_true.extend(labels.cpu().numpy())
      y_pred.extend(predicted.cpu().numpy())

  print("Electra model results:")
  print_results(y_true, y_pred)

  return y_true, y_pred

In [ ]:
y_true, y_pred = evaluate(val_loader)


In [ ]:
results_df = results_df.rename(columns={'Unnamed: 0': 'Models'})

In [ ]:
results_df

In [ ]:
results_df = results_df.set_index(['Models'])

In [ ]:
store_results('Electra Model', y_true, y_pred, target_names)

In [ ]:
results_df

In [ ]:
results_df.to_csv("/content/drive/MyDrive/Cheating Detection for AI generated/Results of models.csv")

## ConVOBert

In [ ]:

PRE_TRAINED_MODEL_NAME_CONVO = 'YituTech/conv-bert-base'
MAX_LEN = 512 # (TODO: change to 128? or something else?)
BATCH_SIZE = 16
NUM_CLASSES = 2

tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME_CONVO, do_lower_case = True)

# Tokenize and encode the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=MAX_LEN)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=MAX_LEN)

train_dataset = EssayDataset(train_encodings, train_labels)
val_dataset = EssayDataset(val_encodings, val_labels)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = AutoModelForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME_CONVO,
    num_labels = NUM_CLASSES,
    output_attentions = False,
    output_hidden_states = False,
)

# we don't need this really: model.classifier = nn.Linear(in_features=model.config.hidden_size, out_features=NUM_CLASSES)

model = model.to(device)


model.train()  # Set the model to training mode
'''

for epoch in trange(num_epochs): # trange - used to make our loops show a smart progress meter
    time_str = datetime.now()
    print(time_str)
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        #loss = outputs.loss (this line was before using loss_func)
        logits = outputs.logits
        loss = loss_func(logits, labels)

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Loss: {avg_loss:.4f}')

'''

In [ ]:
from transformers import TrainingArguments, Trainer, AdamW

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred.predictions, eval_pred.label_ids
    preds = logits.argmax(axis=1)
    kappa = cohen_kappa_score(labels, preds, weights='quadratic')
    return {"cohen_kappa": kappa}

In [ ]:
training_args = TrainingArguments(
  output_dir="./models/model_electra2_prompt_4",
  num_train_epochs=4,  # 1 (1 epoch gives slightly lower accuracy)
  overwrite_output_dir=True,
  evaluation_strategy="epoch",
  per_device_train_batch_size=32,
  per_device_eval_batch_size=32,
  save_total_limit = 2,
  save_strategy = 'epoch',
  load_best_model_at_end=False ) # Make sure all batches are of equal size)
# Instantiate the Trainer class
trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics)
trainer.train()

In [ ]:
pred, actual, _ = trainer.predict(val_dataset)
pred_labels = np.argmax(pred, axis=1)


In [ ]:
store_results('ConvoBert Model', actual, pred_labels, target_names)

In [ ]:
results_df.to_csv('')

In [ ]:
y_true, y_pred = evaluate(val_loader)